In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import datasets
from transformers import DataCollatorForTokenClassification
from transformers import TFAutoModelForTokenClassification
from transformers import create_optimizer
import tensorflow as tf
import evaluate
import numpy as np
from seqeval.metrics import classification_report as seqeval_classification_report
import pandas as pd
from collections import Counter
import random
import os
import re

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

c:\Users\Juliu\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [72]:
import re

def parse_annotations(file1_path, file2_path, file3_path):
    entities = []

    # Parse the first file
    with open(file1_path, "r") as f1:
        for line in f1:
            if line.startswith("T"):  # Entity line
                parts = line.strip().split("\t")
                tag_number, entity_info, text = parts
                entity_type, *offsets = entity_info.split(" ")

                # Handle multiple offset ranges
                offset_ranges = " ".join(offsets).split(";")
                for offset_range in offset_ranges:
                    try:
                        start_offset, end_offset = map(int, offset_range.split(" "))
                        entities.append([start_offset, end_offset, entity_type])
                    except ValueError:
                        continue

    # Parse the second file and add mappings
    with open(file2_path, "r") as f2:
        for line in f2:
            if line.startswith("TT"):
                parts = re.split(r'[\t;\+" "]', line)
                for entry in parts:
                    if entry.isdigit():
                        entry = int(entry)
                        for count, tag in enumerate(entities):
                            if entry == tag[0]:
                                entities[count].append(parts[1])

    # Parse the third file and add entries
    with open(file3_path, "r") as f3:
        for line in f3:
            if line.startswith("TT"):
                parts = line.strip().split("|")  # Split by tab
                print(parts)
                offset = parts[-1].split("\t")[-2].split(" ")
                
                if len(parts) > 1:
                    for entry in offset:
                        if entry.isdigit():

                            entry = int(entry)
                            for count, tag in enumerate(entities):
                                if entry == tag[0]:
                                    entities[count].append(parts[1])
                            break
    entities = list(map(list, set(map(tuple, entities))))
    return entities

print(parse_annotations("cadecv2/original/ARTHROTEC.105.ann", 
                        "cadecv2/meddra/ARTHROTEC.105.ann", 
                        "cadecv2/sct/ARTHROTEC.105.ann"))



['TT1\t300888008 ', ' Swelling of body region ', ' 55 59;60 68\tbody swelling']
['TT8\t278528006 ', ' Facial swelling ', ' 60 68;70 74\tswelling face']
['TT9\t298941006 ', ' Swelling of wrist joint ', ' 60 68;76 82\tswelling wrists']
['TT10\t60728008 ', ' Abdominal swelling ', ' 60 68;84 91\tswelling abdomen']
['TT11\t449614009 ', ' Swelling of lower limb ', ' 60 68;93 99\tswelling thighs']
['TT5\t193462001 ', ' Insomnia ', ' 121 129\tInsomina']
['TT2\t55533009 ', ' Forgetful ', ' + 40917007', 'Confusion', ' 152 179\tforgetfulnes and confussion']
['TT6\t225014007 ', ' Feeling empty ', ' 223 246\t"empty stomach" feeling']
['TT3\t398032003 ', ' Loose stool ', ' 282 294\tloose stools']
['TT7\tCONCEPT_LESS 365 385\tHA (Hyaluronic Acid)']
['TT4\t3384011000036100 ', ' Arthrotec ', ' 397 406\tArthrotec']
[[282, 294, 'ADR', '10024840', ' Loose stool '], [84, 91, 'ADR', '10042679'], [60, 68, 'ADR', '10042674', '10016065', '10042707', '10042679', '10011301', ' Facial swelling ', ' Swelling of wr

In [2]:
def parse_annotations(file1_path, file2_path, file3_path):
    entities = []
    meddra_entities = []
    sct_entities = []

    # Parse the first file
    with open(file1_path, "r") as f1:
        for line in f1:
            if line.startswith("T"):  # Entity line
                parts = line.strip().split("\t")
                tag_number, entity_info, text = parts
                entity_type, *offsets = entity_info.split(" ")
                offset_ranges = " ".join(offsets).split(";")
                for offset_range in offset_ranges:
                    try:
                        start_offset, end_offset = map(int, offset_range.split(" "))
                        entities.append([start_offset, end_offset, entity_type])
                    except ValueError:
                        continue

    # Parse the second file for MedDRA mappings
    if os.path.exists(file2_path):
        with open(file2_path, "r") as f2:
            for line in f2:
                if line.startswith("TT"):
                    parts = re.split(r'[\t;\+\" "]', line)
                    for entry in parts:
                        if entry.isdigit():
                            entry = int(entry)
                            for tag in entities:
                                if entry == tag[0]:
                                    meddra_entities.append(tag + [parts[1]])

    # Parse the third file for SCT mappings
    if os.path.exists(file3_path):
        with open(file3_path, "r") as f3:
            for line in f3:
                if line.startswith("TT"):
                    parts = line.strip().split("|")
                    try:
                        offset = parts[-1].split("\t")[-2].split(" ")
                    except Exception:
                        continue
                    if len(parts) > 1:
                        for entry in offset:
                            if entry.isdigit():
                                entry = int(entry)
                                for tag in entities:
                                    if entry == tag[0]:
                                        sct_entities.append(tag + [parts[1]])

    entities = list(map(list, set(map(tuple, entities))))
    meddra_entities = list(map(list, set(map(tuple, meddra_entities))))
    sct_entities = list(map(list, set(map(tuple, sct_entities))))

    return entities, meddra_entities, sct_entities

def create_iob_labels(text, entities):
    labels = ["O"] * len(text)
    label_details = [None] * len(text)

    for start, end, entity_type, *potential_labels in entities:
        primary_label = entity_type
        potential_ids = potential_labels if potential_labels else []
        labels[start] = f"B-{primary_label}"
        label_details[start] = potential_ids

        for i in range(start + 1, end):
            labels[i] = f"I-{primary_label}"
            label_details[i] = potential_ids

    return labels, label_details

def tokenize_and_label(text, labels, label_details):
    token_labels = []
    text_index = 0

    while text_index < len(text):
        if text[text_index].isspace():
            text_index += 1
            continue

        if re.match(r"\W", text[text_index]):
            label = labels[text_index]
            token_detail = label_details[text_index]
            detail = f"\t{' '.join(token_detail)}" if token_detail else ""
            token_labels.append((text[text_index], label, detail))
            text_index += 1
        else:
            end_index = text_index
            while end_index < len(text) and re.match(r"\w", text[end_index]):
                end_index += 1

            token = text[text_index:end_index]
            token_label = labels[text_index:end_index]
            label = token_label[0] if token_label else "O"
            token_detail = label_details[text_index]
            detail = f"\t{' '.join(token_detail)}" if token_detail else ""
            token_labels.append((token, label, detail))
            text_index = end_index

    return token_labels

def write_to_file(output_path, token_labels):
    with open(output_path, "w") as f:
        for token, label, detail in token_labels:
            # Write formatted token, label, and detail
            detail_str = f"\t{detail}" if detail else ""
            f.write(f"{token}\t{label}{detail_str}\n")
            if token == ".":
                f.write("\n")

# Define paths
base_path = "cadecv2"
original_path = os.path.join(base_path, "original")
text_path = os.path.join(base_path, "text")
meddra_path = os.path.join(base_path, "meddra")
sct_path = os.path.join(base_path, "sct")
output_org_path = os.path.join("OutputFolder", "trainOrg.txt")
output_meddra_path = os.path.join("OutputFolder", "trainMeddra.txt")
output_sct_path = os.path.join("OutputFolder", "trainSct.txt")
output_all_labels_path = os.path.join("OutputFolder", "trainAll.txt")

output_org_lines = []
output_meddra_lines = []
output_sct_lines = []
output_all_labels_lines = []

for text_file in os.listdir(text_path):
    text_file_path = os.path.join(text_path, text_file)
    annotation_file_path = os.path.join(original_path, text_file.replace(".txt", ".ann"))
    meddra_mapping_path = os.path.join(meddra_path, text_file.replace(".txt", ".ann"))
    sct_mapping_path = os.path.join(sct_path, text_file.replace(".txt", ".ann"))

    if os.path.exists(annotation_file_path):
        with open(text_file_path, "r") as f:
            text = f.read()

        entities, meddra_entities, sct_entities = parse_annotations(annotation_file_path, meddra_mapping_path, sct_mapping_path)

        # Process Org labels
        labels, label_details = create_iob_labels(text, entities)
        token_labels_original = tokenize_and_label(text, labels, label_details)
        output_org_lines.extend(token_labels_original)

        # Process MedDRA labels
        labels, label_details = create_iob_labels(text, meddra_entities)
        token_labels_meddra = tokenize_and_label(text, labels, label_details)
        output_meddra_lines.extend(token_labels_meddra)

        # Process SCT labels
        labels, label_details = create_iob_labels(text, sct_entities)
        token_labels_sct = tokenize_and_label(text, labels, label_details)
        output_sct_lines.extend(token_labels_sct)

# Write trainOrg.txt
write_to_file(output_org_path, output_org_lines)
write_to_file(output_meddra_path, output_meddra_lines)
write_to_file(output_sct_path, output_sct_lines)

# Make trainAll.txt identical to trainMeddra.txt
import shutil
shutil.copy(output_meddra_path, output_all_labels_path)


'OutputFolder\\trainAll.txt'

In [3]:
# Open the source and target files
with open('OutputFolder/trainSct.txt', 'r') as source_file, open('OutputFolder/trainAll.txt', 'r+') as target_file:
    # Read all lines from trainAll.txt into a list
    all_lines = target_file.readlines()
    
    # Loop through each line in the source file
    for count, line in enumerate(source_file):
        # Remove trailing whitespace and split the line into entries
        entries = line.strip().split()
        labels = []
        
        # Collect labels starting from the 3rd entry
        for entry in entries[2:]:
            labels.append(entry)
        
        # Ensure we're within bounds for trainAll.txt
        if count < len(all_lines):
            # Add a tab and then the labels to the corresponding line in trainAll.txt
            all_lines[count] = all_lines[count].strip() + '\t' + ' '.join(labels) + '\n'
        else:
            print(f"Warning: Line {count + 1} in trainMeddra.txt exceeds trainAll.txt lines.")
    
    # Move to the beginning of the file and write the updated lines
    target_file.seek(0)
    target_file.writelines(all_lines)
    target_file.truncate()

In [4]:
from collections import Counter

# Define the input file path
input_file_path = "OutputFolder/trainOrg.txt"  # Replace with your file path

# Initialize a counter for labels
label_counts = Counter()

# Processing the file to count labels
with open(input_file_path, "r") as file:
    for line in file:
        line = line.strip()
        if "\t" in line:  # Process only lines with a tab (word-label pairs)
            _, label = line.split("\t")
            label_counts[label] += 1

# Display the counts for each label
print("Label counts:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Label counts:
O: 102670
B-ADR: 6777
I-ADR: 9143
B-Drug: 1802
B-Disease: 285
B-Symptom: 289
I-Symptom: 277
I-Disease: 206
I-Drug: 237
B-Finding: 452
I-Finding: 414


In [7]:
import re

# Normalize file by standardizing line endings and cleaning whitespace
def normalize_file(input_path, output_path, encoding='ISO-8859-1'):
    """
    Normalize a text file by standardizing line endings and removing extra whitespace.
    """
    with open(input_path, 'r', encoding=encoding) as infile:
        content = infile.read()
    
    # Normalize line endings and remove trailing/leading spaces
    content = content.replace('\r\n', '\n').replace('\r', '\n')
    content = re.sub(r'[ \t]+$', '', content, flags=re.MULTILINE)  # Remove trailing spaces
    content = re.sub(r'\n{3,}', '\n\n', content)  # Replace multiple empty lines with a double newline
    content = content.strip()
    
    with open(output_path, 'w', encoding=encoding) as outfile:
        outfile.write(content + '\n')

# Normalize both files
reference_normalized = 'OutputFolder/trainOrg_normalized.txt'
target_normalized = 'OutputFolder/trainAll_normalized.txt'

normalize_file('Outputfolder/trainOrg.txt', reference_normalized)
normalize_file('OutputFolder/trainAll.txt', target_normalized)

# Check sentence counts again
def count_sentences(file_path, encoding='ISO-8859-1'):
    with open(file_path, 'r', encoding=encoding) as file:
        content = file.read()
    return len(content.strip().split('\n\n'))

org_sentence_count = count_sentences(reference_normalized)
all_sentence_count = count_sentences(target_normalized)

print(f"Reference file sentences: {org_sentence_count}")
print(f"Target file sentences: {all_sentence_count}")

# Align normalized files
def align_sentences_token_by_token(reference_path, target_path, output_path, encoding='ISO-8859-1'):
    """
    Align sentences at a token level to ensure both line and sentence consistency.
    """
    with open(reference_path, 'r', encoding=encoding) as ref_file, open(target_path, 'r', encoding=encoding) as tgt_file:
        ref_sentences = ref_file.read().strip().split('\n\n')
        tgt_sentences = tgt_file.read().strip().split('\n\n')

    if len(ref_sentences) != len(tgt_sentences):
        raise ValueError("Sentence counts still do not match after normalization. Please inspect the files.")

    aligned_sentences = []

    for i, (ref_sentence, tgt_sentence) in enumerate(zip(ref_sentences, tgt_sentences)):
        ref_tokens = [line.strip() for line in ref_sentence.split('\n') if line.strip()]
        tgt_tokens = [line.strip() for line in tgt_sentence.split('\n') if line.strip()]

        if len(ref_tokens) != len(tgt_tokens):
            print(f"❗ Token count mismatch in sentence {i+1}: Expected {len(ref_tokens)} tokens, got {len(tgt_tokens)} tokens")
            # Fallback alignment
            aligned_sentence = '\n'.join(tgt_tokens[:len(ref_tokens)])
        else:
            aligned_sentence = '\n'.join(tgt_tokens)
        
        aligned_sentences.append(aligned_sentence)

    with open(output_path, 'w', encoding=encoding) as out_file:
        out_file.write('\n\n'.join(aligned_sentences) + '\n')

    print(f"Aligned file saved to {output_path}")


# Final Alignment
output_file = 'OutputFolder/aligned_trainAll_normalized.txt'
align_sentences_token_by_token(reference_normalized, target_normalized, output_file)


Reference file sentences: 7520
Target file sentences: 7520
Aligned file saved to OutputFolder/aligned_trainAll_normalized.txt


In [9]:
import random
import os

# Create output folder if it doesn't exist
output_folder = 'Train-test-split'
os.makedirs(output_folder, exist_ok=True)

# Define file paths
files = {
    "Org": "OutputFolder/trainOrg.txt",
    "Meddra": "OutputFolder/trainMeddra.txt",
    "Sct": "OutputFolder/trainSct.txt",
    "All": "OutputFolder/aligned_trainAll_normalized.txt"
}

# Define output files in Train-test-split folder
output_files = {
    "train": {key: os.path.join(output_folder, f"train_{key}.txt") for key in files},
    "test": os.path.join(output_folder, "test_Org.txt"),
    "validation": os.path.join(output_folder, "validation_Org.txt")
}

# Helper function to read sentences from a file
def read_sentences(filepath):
    with open(filepath, 'r') as file:
        content = file.read().strip()
    # Normalize line endings and split sentences by double newlines
    content = content.replace('\r\n', '\n').strip()
    sentences = [sentence.strip() for sentence in content.split('\n\n') if sentence.strip()]
    return sentences

# Count sentences in all files
for key, path in files.items():
    sentences = read_sentences(path)
    print(f"{key}: {len(sentences)} sentences")

# Helper function to write sentences to a file
def write_sentences(filepath, sentences):
    with open(filepath, 'w') as file:
        file.write('\n\n'.join(sentences) + '\n')

# Step 1: Read sentences from all files
sentences = {key: read_sentences(files[key]) for key in files}

# Check consistency
sentence_count = len(sentences['Org'])
assert all(len(sentences[key]) == sentence_count for key in sentences), "Sentence counts are inconsistent across files!"

# Step 2: Randomly split indices into train, test, and validation
indices = list(range(sentence_count))
random.shuffle(indices)

train_ratio, test_ratio, val_ratio = 0.8, 0.1, 0.1
train_end = int(train_ratio * sentence_count)
test_end = train_end + int(test_ratio * sentence_count)

train_idx = indices[:train_end]
test_idx = indices[train_end:test_end]
val_idx = indices[test_end:]

# Step 3: Write to output files

# Write train splits (same sentences for all four train files)
for key in files:
    train_sentences = [sentences[key][i] for i in train_idx]
    write_sentences(output_files['train'][key], train_sentences)

# Write test and validation splits (only for Org)
test_sentences = [sentences['Org'][i] for i in test_idx]
validation_sentences = [sentences['Org'][i] for i in val_idx]

write_sentences(output_files['test'], test_sentences)
write_sentences(output_files['validation'], validation_sentences)

print(f"✅ Train/Test/Validation split completed successfully! Files saved in '{output_folder}'")


Org: 7520 sentences
Meddra: 7520 sentences
Sct: 7520 sentences
All: 7520 sentences
✅ Train/Test/Validation split completed successfully! Files saved in 'Train-test-split'
